In [ ]:
import requests

In [ ]:
standings_url = "https://fbref.com/en/comps/9/2023-2024/2023-2024-Premier-League-Stats"

In [ ]:
data=requests.get(standings_url)

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(data.text)

In [ ]:
standings_table = soup.select('table.stats_table')[0]

In [ ]:
links = standings_table.find_all('a')

In [ ]:
links = [l.get('href') for l in links]

In [ ]:
links = [l for l in links if ('/squads/') in l]

In [ ]:
links

In [ ]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [ ]:
team_urls

In [ ]:
team_url = team_urls[0]

In [ ]:
data = requests.get(team_url)

In [ ]:
import pandas as pd 
matches = pd.read_html(data.text, match = "Scores & Fixtures")

In [ ]:
matches[0]

In [ ]:
soup = BeautifulSoup(data.text)


In [ ]:
links = soup.find_all('a')

In [ ]:
links = [l.get('href') for l in links]

In [ ]:
links = [l for l in links if l and ('all_comps/shooting/') in l]

In [ ]:
links

In [ ]:
data = requests.get(f"https://fbref.com{links[0]}")

In [ ]:
shooting = pd.read_html(data.text, match = "Shooting")[0]

In [ ]:
shooting.head()

In [ ]:
shooting.columns = shooting.columns.droplevel()

In [ ]:
shooting.head()

In [ ]:
team_data = matches[0].merge(shooting[["Date","Sh","SoT","Dist","FK","PK","PKatt","xG"]], on = "Date")

In [ ]:
team_data.shape

In [ ]:
years = list(range(2024,2020,-1))

In [ ]:
years

In [ ]:
all_matches = []

In [ ]:
standings_url = "https://fbref.com/en/comps/9/2023-2024/2023-2024-Premier-League-Stats"

In [ ]:
import time, random
for year in years: 
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    league_tables = soup.find_all('table')
    league_table = league_tables[0]
    links = league_table.find_all('a')
    links = [l.get('href') for l in links]
    links = [l for l in links if ('/squads/') in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    prev_season = soup.select("a.prev")[0].get('href')
    standings_url = f"https://fbref.com{prev_season}"

    
    for team_url in team_urls: 
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-"," ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match ='Scores & Fixtures')[0]

        soup = BeautifulSoup(data.text)
        links = [l.get('href') for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text,match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[['Date','Sh','SoT','Dist','FK','PK','PKatt']], on = "Date")
        except ValueError: 
            continue

        team_data = team_data[team_data["Comp"] == 'Premier League']
        team_data['Season'] = year
        team_data['Team'] = team_name
        all_matches.append(team_data)
        time.sleep(random.uniform(6, 10))

    

In [ ]:
match_df = pd.concat(all_matches)

In [ ]:
match_df.to_csv('matches_full.csv')